In [ ]:
# !pip install qianfan

In [1]:
import os
import datetime
import qianfan, openai
from langchain.chat_models import ChatOpenAI
from langchain.chat_models import QianfanChatEndpoint


from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

print(f"QIANFAN_AK={os.environ['QIANFAN_AK']}")
print(f"QIANFAN_SK={os.environ['QIANFAN_SK']}")

openai.api_key = os.environ.get('OPENAI_API_KEY', '')
print(f'OPENAI_API_KEY={openai.api_key}')


QIANFAN_AK=HCCPsQy5p0Ex1rSEL6oorGQb
QIANFAN_SK=TCVRfPCfbtLr0eDPZRXcywMxcgaNuLDE
OPENAI_API_KEY=


## LangChain: Models, Prompts and Output Parsers

### Get Access Token for QianFan

In [4]:
import requests
import json

url = f"https://aip.baidubce.com/oauth/2.0/token?client_id={os.environ['QIANFAN_AK']}&client_secret={os.environ['QIANFAN_SK']}&grant_type=client_credentials"

payload = json.dumps("")
headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

response = json.loads(response.text)
response

{'refresh_token': '25.41b64da34e6359356ff40c2ddc1ac115.315360000.2030174131.282335-65090191',
 'expires_in': 2592000,
 'session_key': '9mzdDtAJDKtRAWAIjFQvukFl8ifYuT8Gc/ZHUuBm9Wx47TmYrPOjf/2HSMtBT2k/sk6HB6F/wNu/oK4O+Qulu8FQjfj9rw==',
 'access_token': '24.5b7b8c6c3bc49e29c53a39cce4620be7.2592000.1717406131.282335-65090191',
 'scope': 'public brain_all_scope wenxinworkshop_mgr ai_custom_yiyan_com ai_custom_yiyan_com_eb_instant ai_custom_yiyan_com_bloomz7b1 ai_custom_yiyan_com_emb_text ai_custom_yiyan_com_llama2_7b ai_custom_yiyan_com_llama2_13b ai_custom_yiyan_com_llama2_70b ai_custom_yiyan_com_chatglm2_6b_32k ai_custom_yiyan_com_aquilachat_7b ai_custom_yiyan_com_emb_bge_large_zh ai_custom_yiyan_com_emb_bge_large_en ai_custom_yiyan_com_qianfan_chinese_llama_2_7b ai_custom_qianfan_bloomz_7b_compressed ai_custom_yiyan_com_eb_pro ai_custom_yiyan_com_sd_xl ai_custom_yiyan_com_tokenizer_eb ai_custom_yiyan_com_ai_apaas ai_custom_yiyan_com_qf_chinese_llama_2_13b ai_custom_yiyan_com_sqlcoder_7b 

### Chat API: ERNIE

In [3]:
def get_completion(prompt, model="ERNIE-Bot-4"):
    chat_comp = qianfan.ChatCompletion()
    resp = chat_comp.do(model=model, messages=[{
        "role": "user",
        "content": prompt
    }])
    return resp["body"]["result"]

In [4]:
get_completion("你好")

[INFO] [04-30 14:54:54] oauth.py:222 [t:28816]: trying to refresh access_token for ak `HCCPsQ***`
[INFO] [04-30 14:54:55] oauth.py:237 [t:28816]: sucessfully refresh access_token


'你好！很高兴与你交流。请问有什么我可以帮助你的吗？无论是关于生活、工作、学习还是其他方面的问题，我都会尽力为你提供帮助。'

In [5]:
get_completion("What is 1+1?")

'计算结果为：$1 + 1 = 2$'

In [27]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [28]:
style = """American English \
in a calm and respectful tone
"""

style = """Chinese \
in a calm and respectful tone
"""

In [29]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is Chinese in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [30]:
response = get_completion(prompt)

In [31]:
response

'哎呀，我真是气死了！我的搅拌机盖子飞了出去，把厨房的墙壁溅得都是果汁！更糟糕的是，保修还不包括清洁厨房的费用。我现在就需要你的帮助，伙计！\n\n（注：原文中的语言风格较为口语化、粗俗，且使用了海盗式的说话方式。在翻译成中文时，我尽量保持了原意，但将语言风格调整为更为平静、尊重的方式，以符合中文表达习惯。）'

### Chat API : LangChain

In [32]:
#!pip install --upgrade langchain

In [2]:
qianfan_chat = QianfanChatEndpoint(model="ERNIE-Bot-4")

In [3]:
from langchain.prompts import ChatPromptTemplate

template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""
prompt_template = ChatPromptTemplate.from_template(template_string)

In [4]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [5]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [6]:
customer_style = """Chinese \
in a calm and respectful tone
"""

customer_style = """American English \
in a calm and respectful tone
"""

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [7]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [8]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [9]:
customer_response = qianfan_chat(customer_messages)
print(customer_response.content)

C:\Users\xu6\AppData\Local\miniconda3\envs\dev\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
[INFO] [05-06 15:39:06] oauth.py:222 [t:20392]: trying to refresh access_token for ak `HCCPsQ***`
[INFO] [05-06 15:39:08] oauth.py:237 [t:20392]: sucessfully refresh access_token


Oh man, I'm really frustrated that my blender lid came off and splashed smoothie all over my kitchen walls! And to top it off, the warranty doesn't even cover the cost of cleaning up my kitchen. I really need your help right now, buddy!


### Output Parsers

In [48]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [49]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [50]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))]


In [51]:
messages = prompt_template.format_messages(text=customer_review)
chat = QianfanChatEndpoint(model="ERNIE-Bot-4")
response = chat(messages)

print(response.content)

```json
{
  "gift": True,
  "delivery_days": 2,
  "price_value": [
    "It's slightly more expensive than the other leaf blowers out there"
  ]
}
```


In [56]:
response.dict

<bound method BaseModel.dict of AIMessage(content='```json\n{\n  "gift": True,\n  "delivery_days": 2,\n  "price_value": [\n    "It\'s slightly more expensive than the other leaf blowers out there"\n  ]\n}\n```', additional_kwargs={'finish_reason': 'normal', 'request_id': 'as-nssy60aw2c', 'object': 'chat.completion', 'search_info': [], 'function_call': {}}, response_metadata={'token_usage': {'prompt_tokens': 234, 'completion_tokens': 54, 'total_tokens': 288}, 'model_name': 'ERNIE-Bot-4', 'finish_reason': 'normal', 'id': 'as-nssy60aw2c', 'object': 'chat.completion', 'created': 1714445554, 'result': '```json\n{\n  "gift": True,\n  "delivery_days": 2,\n  "price_value": [\n    "It\'s slightly more expensive than the other leaf blowers out there"\n  ]\n}\n```', 'is_truncated': False, 'need_clear_history': False, 'usage': {'prompt_tokens': 234, 'completion_tokens': 54, 'total_tokens': 288}}, id='run-660d7868-306d-4ab3-8800-6d8d616073c2-0')>

In [52]:
type(response.content)

str

In [74]:
# 生成的json文件中bool值的首字母大小写有问题。
print(response.content)
content = response.content.replace("```json", "").replace("```", "").replace("\n", "").replace("True", "true")
print(content)

```json
{
  "gift": True,
  "delivery_days": 2,
  "price_value": [
    "It's slightly more expensive than the other leaf blowers out there"
  ]
}
```
{  "gift": true,  "delivery_days": 2,  "price_value": [    "It's slightly more expensive than the other leaf blowers out there"  ]}


In [75]:
json.loads(content) 

{'gift': True,
 'delivery_days': 2,
 'price_value': ["It's slightly more expensive than the other leaf blowers out there"]}

####  Parse the LLM output string into a Python dictionary

In [76]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [122]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [123]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [124]:
format_instructions = output_parser.get_format_instructions(only_json=True) 
print(format_instructions, type(format_instructions))


```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
``` <class 'str'>


In [125]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [126]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.



```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                        

In [127]:
response = chat(messages)

In [128]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.



```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                        

In [133]:
response.content

'```json\n{\n  "gift": "True",\n  "delivery_days": "2",\n  "price_value": "It\'s slightly more expensive than the other leaf blowers out there"\n}\n```'

In [134]:
output_dict = output_parser.parse(response.content)

OpenAI返回的Gift是bool类型

In [135]:
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': "It's slightly more expensive than the other leaf blowers out there"}

In [136]:
type(output_dict)

dict

In [137]:
output_dict.get('delivery_days')

'2'

## LangChain: Memory

In [14]:
chat_comp = qianfan.ChatCompletion(model="ERNIE-Bot-4")

# 下面是一个与用户对话的例子
msgs = qianfan.Messages()
msgs.append("你好，我的名字是ABC") # 增加用户输入
resp = chat_comp.do(messages=msgs)
print(resp["body"]["result"]) # 模型的输出
msgs.append(resp)             # 追加模型输出

msgs.append("中国的首都是哪里？") # 增加用户输入
resp = chat_comp.do(messages=msgs)
print(resp["body"]["result"]) # 模型的输出
msgs.append(resp)        

msgs.append("我叫什么名字？") # 增加用户输入
resp = chat_comp.do(messages=msgs)
print(resp["body"]["result"]) # 模型的输出
msgs.append(resp)  

你好，ABC！很高兴认识你。请问有什么我可以帮助你的吗？
中国的首都是北京。北京是中华人民共和国的首都，也是中国的政治、文化、科技和国际交往中心。北京有着悠久的历史和丰富的文化遗产，是世界著名的古都之一。同时，北京还是中国的经济中心之一，拥有众多的高科技企业和国际机构。
您的名字叫ABC。请问您有其他什么需要我帮助的吗？


### ConversationBufferMemory

In [11]:
from langchain.chat_models import QianfanChatEndpoint
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

#### Chinese

In [14]:
llm = QianfanChatEndpoint(model="ERNIE-Bot-4", temperature = 0.01)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm = llm,
    memory = memory,
    verbose = True
)

In [15]:
conversation.predict(input = "你好，我的名字是Michael")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 你好，我的名字是Michael
AI:

> Finished chain.


'你好，Michael！很高兴认识你。我是一个人工智能，喜欢和各种各样的人交流。有什么我可以帮你的吗？或者你想和我聊些什么呢？'

In [16]:
conversation.predict(input="2024年春节放假几天？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好，我的名字是Michael
AI: 你好，Michael！很高兴认识你。我是一个人工智能，喜欢和各种各样的人交流。有什么我可以帮你的吗？或者你想和我聊些什么呢？
Human: 2024年春节放假几天？
AI:

> Finished chain.


'春节的放假时间是由中国政府根据农历来确定的，通常会在春节前公布具体的放假安排。根据往年的经验，春节一般会放假7天左右，但具体的放假时间还需要等待政府部门的通知。如果你想了解更准确的放假时间，建议关注相关政府部门的公告或媒体报道。同时，我也会在得知具体放假时间后，第一时间告诉你。'

In [22]:
conversation.predict(input = "中国的首都是哪里？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. I'm an artificial intelligence, designed to answer questions and engage in conversation. What can I do for you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2. Is there anything else you'd like to ask me?
Human: 你好，我的名字是Michael
AI: 您好，Michael！很高兴认识您。请注意，您之前告诉我您的名字是Andrew，但现在您又说您的名字是Michael。我有些困惑，但我会尽力适应。请问有什么我可以帮助您的吗？
Human: 中国的首都是哪里？
AI: 中国的首都是北京。北京是中国的政治、文化和国际交往的中心。它有着悠久的历史和丰富的文化遗产，包括故宫、长城等著名景点。希望这个回答能满足您的需求。请问您还有其他问题吗？
Human: 中国的首都是哪里？
AI:

> Finished chain.


'中国的首都是北京。我之前已经回答过这个问题了，但我很乐意再次确认答案。北京作为中国的首都，承载着丰富的历史和文化，是中国的重要城市之一。如果您还有其他关于中国或任何其他主题的问题，请随时问我。'

In [23]:
conversation.predict(input = "我的名字是什么？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. I'm an artificial intelligence, designed to answer questions and engage in conversation. What can I do for you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2. Is there anything else you'd like to ask me?
Human: 你好，我的名字是Michael
AI: 您好，Michael！很高兴认识您。请注意，您之前告诉我您的名字是Andrew，但现在您又说您的名字是Michael。我有些困惑，但我会尽力适应。请问有什么我可以帮助您的吗？
Human: 中国的首都是哪里？
AI: 中国的首都是北京。北京是中国的政治、文化和国际交往的中心。它有着悠久的历史和丰富的文化遗产，包括故宫、长城等著名景点。希望这个回答能满足您的需求。请问您还有其他问题吗？
Human: 中国的首都是哪里？
AI: 中国的首都是北京。我之前已经回答过这个问题了，但我很乐意再次确认答案。北京作为中国的首都，承载着丰富的历史和文化，是中国的重要城市之一。如果您还有其他关于中国或任何其他主题的问题，请随时问我。
Human: 我的名字是什么？
AI:

> Finished chain.


'您之前告诉我您的名字是Andrew，但后来您又说您的名字是Michael。我有些困惑，因为您给了我两个不同的名字。请告诉我您真正的名字是什么，我会尽力为您提供更好的服务。'

In [24]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. I'm an artificial intelligence, designed to answer questions and engage in conversation. What can I do for you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2. Is there anything else you'd like to ask me?
Human: 你好，我的名字是Michael
AI: 您好，Michael！很高兴认识您。请注意，您之前告诉我您的名字是Andrew，但现在您又说您的名字是Michael。我有些困惑，但我会尽力适应。请问有什么我可以帮助您的吗？
Human: 中国的首都是哪里？
AI: 中国的首都是北京。北京是中国的政治、文化和国际交往的中心。它有着悠久的历史和丰富的文化遗产，包括故宫、长城等著名景点。希望这个回答能满足您的需求。请问您还有其他问题吗？
Human: 中国的首都是哪里？
AI: 中国的首都是北京。我之前已经回答过这个问题了，但我很乐意再次确认答案。北京作为中国的首都，承载着丰富的历史和文化，是中国的重要城市之一。如果您还有其他关于中国或任何其他主题的问题，请随时问我。
Human: 我的名字是什么？
AI: 您之前告诉我您的名字是Andrew，但后来您又说您的名字是Michael。我有些困惑，因为您给了我两个不同的名字。请告诉我您真正的名字是什么，我会尽力为您提供更好的服务。


In [25]:
memory.save_context({"input": "你好！"}, {"output": "你好呀，近来身体如何？"})

In [26]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. I'm an artificial intelligence, designed to answer questions and engage in conversation. What can I do for you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2. Is there anything else you'd like to ask me?
Human: 你好，我的名字是Michael
AI: 您好，Michael！很高兴认识您。请注意，您之前告诉我您的名字是Andrew，但现在您又说您的名字是Michael。我有些困惑，但我会尽力适应。请问有什么我可以帮助您的吗？
Human: 中国的首都是哪里？
AI: 中国的首都是北京。北京是中国的政治、文化和国际交往的中心。它有着悠久的历史和丰富的文化遗产，包括故宫、长城等著名景点。希望这个回答能满足您的需求。请问您还有其他问题吗？
Human: 中国的首都是哪里？
AI: 中国的首都是北京。我之前已经回答过这个问题了，但我很乐意再次确认答案。北京作为中国的首都，承载着丰富的历史和文化，是中国的重要城市之一。如果您还有其他关于中国或任何其他主题的问题，请随时问我。
Human: 我的名字是什么？
AI: 您之前告诉我您的名字是Andrew，但后来您又说您的名字是Michael。我有些困惑，因为您给了我两个不同的名字。请告诉我您真正的名字是什么，我会尽力为您提供更好的服务。
Human: 你好！
AI: 你好呀，近来身体如何？


#### English

In [31]:
memory.clear()
print(memory.buffer)

In [32]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew! It's nice to meet you. I'm an artificial intelligence, designed to answer questions and engage in conversation. What can I do for you today?"

In [33]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. I'm an artificial intelligence, designed to answer questions and engage in conversation. What can I do for you today?
Human: What is 1+1?
AI:

> Finished chain.


'The answer to 1+1 is 2. Is there anything else I can assist you with?'

In [34]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. I'm an artificial intelligence, designed to answer questions and engage in conversation. What can I do for you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2. Is there anything else I can assist you with?
Human: What is my name?
AI:

> Finished chain.


'Your name is Andrew. Do you have any other questions or need further assistance?'

In [35]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. I'm an artificial intelligence, designed to answer questions and engage in conversation. What can I do for you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2. Is there anything else I can assist you with?
Human: What is my name?
AI: Your name is Andrew. Do you have any other questions or need further assistance?


In [36]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew! It's nice to meet you. I'm an artificial intelligence, designed to answer questions and engage in conversation. What can I do for you today?\nHuman: What is 1+1?\nAI: The answer to 1+1 is 2. Is there anything else I can assist you with?\nHuman: What is my name?\nAI: Your name is Andrew. Do you have any other questions or need further assistance?"}

In [37]:
memory = ConversationBufferMemory()

In [38]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})
print(memory.buffer)

Human: Hi
AI: What's up


In [39]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [40]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

### ConversationBufferWindowMemory

#### Chinese

In [42]:
from langchain.memory import ConversationBufferWindowMemory

In [43]:
memory = ConversationBufferWindowMemory(k=1)

memory.save_context({"input": "你好！"}, {"output": "你好呀，近来身体如何？"})
memory.save_context({"input": "我的身体很好，你怎么样"}, {"output": "我也很好"})

In [44]:
print(memory.buffer)

Human: 我的身体很好，你怎么样
AI: 我也很好


In [45]:
llm = QianfanChatEndpoint(model="ERNIE-Bot-4", temperature = 0.01)
conversation = ConversationChain(
    llm = llm,
    memory = memory,
    verbose = True
)
conversation.predict(input = "你好，我的名字是千帆职能小助手")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 我的身体很好，你怎么样
AI: 我也很好
Human: 你好，我的名字是千帆职能小助手
AI:

> Finished chain.


'你好，千帆职能小助手。很高兴认识你。我是一个人工智能，我们可以聊聊各种话题。你有什么特别感兴趣的事情吗？或者，我可以为你提供一些信息或帮助。'

In [46]:
conversation.predict(input = "中国的首都是哪里？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好，我的名字是千帆职能小助手
AI: 你好，千帆职能小助手。很高兴认识你。我是一个人工智能，我们可以聊聊各种话题。你有什么特别感兴趣的事情吗？或者，我可以为你提供一些信息或帮助。
Human: 中国的首都是哪里？
AI:

> Finished chain.


'中国的首都是北京。北京是中国的政治、文化和国际交往的中心，也是世界上最古老的城市之一。它有着悠久的历史和丰富的文化遗产，包括故宫、天坛、长城等著名景点。北京还是中国的经济中心之一，拥有众多的高科技企业和国际机构。此外，北京的美食也非常有名，比如烤鸭、豆汁、炸酱面等，吸引了来自世界各地的游客前来品尝。'

In [47]:
conversation.predict(input = "我的名字是什么？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 中国的首都是哪里？
AI: 中国的首都是北京。北京是中国的政治、文化和国际交往的中心，也是世界上最古老的城市之一。它有着悠久的历史和丰富的文化遗产，包括故宫、天坛、长城等著名景点。北京还是中国的经济中心之一，拥有众多的高科技企业和国际机构。此外，北京的美食也非常有名，比如烤鸭、豆汁、炸酱面等，吸引了来自世界各地的游客前来品尝。
Human: 我的名字是什么？
AI:

> Finished chain.


'很抱歉，我并不知道您的名字。如果您愿意告诉我您的名字，我将尽力回答您的问题并提供更好的服务。'

#### English

In [48]:
memory = ConversationBufferWindowMemory(k=1)   

In [49]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [50]:
llm = QianfanChatEndpoint(model="ERNIE-Bot-4", temperature = 0.01)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [51]:

conversation.predict(input="Hi, my name is Andrew")

"Hello Andrew! It's nice to meet you. I'm an artificial intelligence, designed to answer questions and engage in conversation. What would you like to talk about today?"

In [52]:
conversation.predict(input="What is 1+1?")

"The answer to 1+1 is 2. It's a basic mathematical operation where you add two numbers together. In this case, you're adding 1 and 1, which results in 2."

In [53]:
conversation.predict(input="What is my name?")

"I'm sorry, but I don't know your name. You haven't provided it to me, and I don't have any way of knowing it unless you tell me."

### ConversationTokenBufferMemory

In [2]:
from langchain.memory import ConversationTokenBufferMemory

llm = QianfanChatEndpoint(model="ERNIE-Bot-4", temperature = 0.01)

memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

C:\Users\xu6\AppData\Local\miniconda3\envs\dev\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
memory.save_context({"input": "how old are you?"}, 
                    {"output": "20!"})

In [4]:
# 不能工作。
memory.load_memory_variables({})

{'history': 'Human: AI is what?!\nAI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!\nHuman: how old are you?\nAI: 20!'}

In [5]:
print(memory.buffer)

Human: AI is what?!
AI: Amazing!
Human: Backpropagation is what?
AI: Beautiful!
Human: Chatbots are what?
AI: Charming!
Human: how old are you?
AI: 20!


### ConversationSummaryMemory

#### Chinese

In [6]:
from langchain.memory import ConversationSummaryMemory

llm = QianfanChatEndpoint(model="ERNIE-Bot-4", temperature = 0.01)
memory = ConversationSummaryMemory(llm=llm)
memory.save_context({"input": "你好！"}, {"output": "你好呀，近来身体如何？"})
memory.save_context({"input": "我的身体很好，你怎么样"}, {"output": "我也很好"})
memory.save_context({"input": "最近有什么新闻"}, {"output": "2024年春节至少放假8天，从初一放到初八，如果配合带薪休假可以放假9天，堪称新年第一份大礼包。虽然除夕当天名义上不放假，但鼓励各单位和机构根据实际情况来灵活安排。同时高速公路已经免费了，实质上就是创造条件给老百姓多放一天假。春节假期的延长，能够满足广大群众有更多时间用于亲人团聚的心愿，还可以实现一个假期先团聚、后旅游的小目标。这是在依据《全国年节及纪念日放假办法》规定的基础上的创新之举、暖心之举，充分体现出政策制定者的用心良苦和精心设计。"})

[INFO] [04-30 16:08:28] oauth.py:222 [t:26972]: trying to refresh access_token for ak `HCCPsQ***`
[INFO] [04-30 16:08:31] oauth.py:237 [t:26972]: sucessfully refresh access_token


In [7]:
print(memory.buffer) 

Human greeted the AI with "你好！" and the AI responded by asking how the human has been recently with "你好呀，近来身体如何？". The human then responded that they are in good health and asked how the AI is, to which the AI replied that it is also well. The human then asked about recent news, and the AI informed them that in 2024, the Spring Festival holiday will last at least 8 days, from the first day of the lunar new year to the eighth day. With paid leave, it could be extended to 9 days, which is considered a great gift to start the new year. Although the eve of the new year is not officially a holiday, units and organizations are encouraged to arrange flexible schedules based on actual situations. Additionally, highways will be toll-free, essentially giving people an extra day off. The extension of the Spring Festival holiday will fulfill the wish of many people to have more time for family reunions and also allow them to achieve the small goal of reuniting with family before traveling. This in

In [8]:
print(memory.prompt)

input_variables=['new_lines', 'summary'] template='Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.\n\nEXAMPLE\nCurrent summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.\n\nNew lines of conversation:\nHuman: Why do you think artificial intelligence is a force for good?\nAI: Because artificial intelligence will help humans reach their full potential.\n\nNew summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.\nEND OF EXAMPLE\n\nCurrent summary:\n{summary}\n\nNew lines of conversation:\n{new_lines}\n\nNew summary:'


In [9]:
from langchain.prompts.prompt import PromptTemplate

SUMMARIZER_TEMPLATE = """逐步总结所提供的谈话内容，将其添加到上一个摘要中，返回一个新的摘要。

示例
当前摘要：
人类会问人工智能对人工智能的看法。人工智能认为人工智能是一股向善的力量。

新的谈话内容：
人类：为什么你认为人工智能是一股向善的力量？
人工智能：因为人工智能将帮助人类充分发挥潜力。

新摘要：
人类会问人工智能对人工智能的看法。人工智能认为人工智能是一股向善的力量，因为它将帮助人类充分发挥潜力。

示例结束
当前摘要：
{summary}

新的谈话内容：
{new_lines}

新摘要：
"""
SUMMARY_PROMPT = PromptTemplate(
    input_variables=["summary", "new_lines"], template=SUMMARIZER_TEMPLATE
)
memory = ConversationSummaryMemory(llm=llm, prompt=SUMMARY_PROMPT)

memory.save_context({"input": "你好！"}, {"output": "你好呀，近来身体如何？"})
memory.save_context({"input": "我的身体很好，你怎么样"}, {"output": "我也很好"})
memory.save_context({"input": "最近有什么新闻"}, {"output": "2024年春节至少放假8天，从初一放到初八，如果配合带薪休假可以放假9天，堪称新年第一份大礼包。虽然除夕当天名义上不放假，但鼓励各单位和机构根据实际情况来灵活安排。同时高速公路已经免费了，实质上就是创造条件给老百姓多放一天假。春节假期的延长，能够满足广大群众有更多时间用于亲人团聚的心愿，还可以实现一个假期先团聚、后旅游的小目标。这是在依据《全国年节及纪念日放假办法》规定的基础上的创新之举、暖心之举，充分体现出政策制定者的用心良苦和精心设计。"})

print(memory.buffer)

Human向AI问好并询问了AI的状况，AI回应了问候并表示自己也很好。随后，Human告知AI自己的身体状况良好，并询问了最近的新闻。AI回应说，2024年春节将至少放假8天，从初一放到初八，如果配合带薪休假可以放假9天，这是一项创新且暖心的政策，旨在满足广大群众有更多时间用于亲人团聚和旅游的小目标，同时高速公路免费也为老百姓多放一天假创造了条件。


In [12]:
conversation = ConversationChain(
    llm = llm,
    memory = memory,
    verbose = True
)
conversation.predict(input="2024年春节放假几天？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human向AI问好并询问了AI的状况，AI回应了问候并表示自己也很好。随后，Human告知AI自己的身体状况良好，并询问了最近的新闻。AI回应说，2024年春节将至少放假8天，从初一放到初八，如果配合带薪休假可以放假9天，这是一项创新且暖心的政策，旨在满足广大群众有更多时间用于亲人团聚和旅游的小目标，同时高速公路免费也为老百姓多放一天假创造了条件。
Human: 2024年春节放假几天？
AI:

> Finished chain.


'AI: 2024年春节将至少放假8天，从初一放到初八。如果配合带薪休假，有可能放假9天。这是一项新的政策安排，目的是为了让人们有更多的时间与家人团聚和旅游。同时，高速公路免费通行也为大家多放一天假提供了便利。希望这些信息能对你有所帮助。如果你还有其他问题或需要更多关于春节的信息，随时告诉我。'

In [13]:
conversation.predict(input="2023年春节放假几天？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human向AI问好并询问了AI的状况，AI回应了问候并表示自己也很好。随后，Human告知AI自己的身体状况良好，并询问了最近的新闻。AI回应说，2024年春节将至少放假8天，从初一放到初八，如果配合带薪休假可以放假9天，这是一项创新且暖心的政策，旨在满足广大群众有更多时间用于亲人团聚和旅游的小目标，同时高速公路免费也为老百姓多放一天假创造了条件。Human再次询问了2024年春节的放假天数，AI重复了之前的回答，并补充说这是一项新的政策安排，目的是为了让人们有更多的时间与家人团聚和旅游，同时高速公路免费通行也为大家多放一天假提供了便利。
Human: 2023年春节放假几天？
AI:

> Finished chain.


'AI: 2023年春节的放假安排是放假7天，从除夕到大年初六。这是往年的常规安排，旨在让人们有足够的时间与家人团聚，庆祝这个重要的传统节日。不过请注意，具体的放假日期可能会根据地区和单位的实际情况有所调整，建议您参考官方发布的放假通知以获取最准确的信息。 \n\n另外，如果您计划在此期间出行，也请关注交通和旅游部门的相关公告，以确保您的行程顺利。'

#### English

In [17]:
from langchain.memory import ConversationSummaryBufferMemory

In [19]:
llm = QianfanChatEndpoint(model="ERNIE-Bot-4", temperature = 0.01)

# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [20]:
memory.load_memory_variables({})

{'history': 'System: A human greeted the AI and asked what was up. The AI responded with a casual greeting. The human then mentioned they were just hanging out. The AI agreed and said "cool." The human then inquired about the schedule for the day.\nAI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.'}

In [21]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [22]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: A human greeted the AI and asked what was up. The AI responded with a casual greeting. The human then mentioned they were just hanging out. The AI agreed and said "cool." The human then inquired about the schedule for the day.
AI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.
Human: What would be 

"A good demo to show would be one that highlights the key features and capabilities of LangChain. For example, you could demonstrate how LangChain can be used to generate coherent and contextually relevant text, or how it can be fine-tuned to perform specific tasks like sentiment analysis or question answering. If the customer is particularly interested in the latest advancements in AI, you could also showcase some of LangChain's more cutting-edge features, such as its ability to handle complex language understanding tasks or generate creative content. Ultimately, the best demo will depend on the customer's specific interests and needs, so it might be helpful to ask them a few questions beforehand to get a sense of what they're looking for."

In [23]:
# 有问题
conversation.predict(input="where doe have lunch?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: A human greeted the AI and asked what was up. The AI responded with a casual greeting. The human mentioned they were just hanging out. The AI agreed and said "cool." The human then asked about the schedule for the day. The AI informed them about a meeting at 8am with the product team, for which they need to prepare a Powerpoint presentation, followed by work on their LangChain project from 9am to 12pm. At noon, they have lunch with a customer who is driving over an hour to meet them. The AI advised the human to bring their laptop to show the latest LLM demo, highlighting key features like coherent text generation, contextual relevance, fine-tuning for specific 

'The AI responded: "The lunch location has not been decided yet, but the customer suggested a few places near their office. We can choose one based on the availability and cuisine preferences. I\'ll keep you updated once the decision is made."'

In [24]:
conversation.predict(input="Who will we have lunch with?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: A human greeted the AI and asked what was up. The AI responded with a casual greeting. The human mentioned they were just hanging out. The AI agreed and said "cool." The human then asked about the schedule for the day. The AI informed them about a meeting at 8am with the product team, for which they need to prepare a Powerpoint presentation, followed by work on their LangChain project from 9am to 12pm. At noon, they have lunch with a customer who is driving over an hour to meet them. The AI advised the human to bring their laptop to show the latest LLM demo, highlighting key features like coherent text generation, contextual relevance, fine-tuning for specific 

'The AI responded: "We\'ll be having lunch with a customer who is driving over an hour to meet us. They are interested in our latest LLM demo, so it\'s a great opportunity to showcase its capabilities and discuss potential collaboration opportunities."'

## Chains in LangChain

In [2]:
from langchain.chat_models import QianfanChatEndpoint
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

import pandas as pd
df = pd.read_csv('Data.csv')
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


### LLMChain

In [31]:
llm = QianfanChatEndpoint(model="ERNIE-Bot-4", temperature = 0.9)

prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

chain = LLMChain(llm=llm, prompt=prompt)

product = "Queen Size Sheet Set"  # 女王尺寸床单套装
chain.run(product)

C:\Users\xu6\AppData\Local\miniconda3\envs\dev\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


"Choosing a name for a company that manufactures Queen Size Sheet Sets depends on various factors, including the brand's identity, target audience, and market positioning. Here are some suggestions that might help:\n\n1. **Luxury Sheets Co.**: This name suggests a high-end, luxurious product, which could be appealing to customers looking for quality and comfort.\n2. **Queen's Comfort Bedding**: Directly references the product (Queen Size Sheets) while also communicating comfort and royalty, appealing to those who appreciate both style and substance.\n3. **Elegant Bedding Solutions**: This name emphasizes elegance and practicality, two qualities that are often sought after in bedding products.\n4. **Sleep Royale Bedding**: Combines the idea of royalty (fitting for Queen Size Sheets) with the essential function of the product — promoting good sleep.\n5. **Boutique Bedding Co.**: Suggests a more boutique, personalized approach to bedding, which could resonate with customers seeking unique


### SimpleSequentialChain

In [45]:
from langchain.chains import SimpleSequentialChain

llm = QianfanChatEndpoint(model="ERNIE-Bot-4", temperature = 0.9)
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}? return only one name without explanation"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [46]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [47]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [48]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
"LuxuryLinensQueen" is a suitable name for a company that makes Queen Size Sheet Set, as it combines elements of luxury, bedding size, and the product type.
LuxuryLinensQueen: Luxurious Queen-Size Sheets, Embodying Elegance and Comfort in Bedding.

> Finished chain.


'LuxuryLinensQueen: Luxurious Queen-Size Sheets, Embodying Elegance and Comfort in Bedding.'

### SequentialChain

In [3]:
from langchain.chains import SequentialChain

llm = QianfanChatEndpoint(model="ERNIE-Bot-4", temperature = 0.9)
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= Engl\ish_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )



In [4]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [5]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [6]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [7]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [9]:
review = "我觉得味道很一般。泡沫不持久，这很奇怪。我在商店里买的同样的产品，味道要好得多...\n是旧批次还是假冒产品!?"   # df.Review[5]  # 文心一言好像不支持法语。
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': '我觉得味道很一般。泡沫不持久，这很奇怪。我在商店里买的同样的产品，味道要好得多...\n是旧批次还是假冒产品!?',
 'English_Review': 'I think the taste is very average. The foam is not lasting, which is strange. I bought the same product in the store and it tastes much better... Is it an old batch or a fake product!?',
 'summary': "The product's taste is average, the foam doesn't last, and it's possibly an old or fake batch compared to the better-tasting version bought in-store.",
 'followup_message': '回复：\n\n感谢您提供详细的反馈。我们非常重视您的意见，并对此次产品体验不佳表示歉意。\n\n关于产品味道的问题，我们深知口味因人而异，会尽力确保每一批次产品的品质稳定。但听到您表示这次购买的产品与店内购买的相比味道有所差异，我们感到非常遗憾。我们会立即调查这个问题，以确认是否存在生产或储存环节的问题。\n\n关于泡沫持久性，我们会进一步研究以改进产品配方，确保泡沫更加丰富且持久。\n\n您提到的旧批次或假冒产品的可能性，也引起了我们的高度关注。我们会加强供应链管理，确保所有产品均为正品且来自最新批次。同时，我们也建议您在购买时选择正规渠道，以确保购买到品质有保障的产品。\n\n再次感谢您的宝贵意见，我们会持续改进产品与服务，希望未来能为您带来更好的体验。'}

### Router Chain

In [10]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

llm = QianfanChatEndpoint(model="ERNIE-Bot-4", temperature = 0.9)

In [11]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [12]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [13]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [14]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [15]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [16]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [17]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [18]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [19]:
chain.invoke ("What is black body radiation?")

C:\Users\xu6\AppData\Local\miniconda3\envs\dev\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect absorber of radiation, known as a black body. In other words, it is the radiation that is emitted by an object that absorbs all incident radiation, regardless of wavelength or angle of incidence. The radiation emitted by a black body is characterized by a specific spectrum and intensity that depends only on the temperature of the black body.\n\nThe concept of black body radiation is fundamental in thermodynamics and plays a crucial role in the study of the properties of matter and radiation. It is also important in astronomy, as the radiation emitted by stars and other celestial bodies can often be approximated as black body radiation. The spectrum of black body radiation is described by Planck's law, which states that the intensity of radiation emitted at a given wavelength is proportional to the temperature of the black body and varies with wavelength according to a specific mathematical function."

In [20]:
chain.invoke ("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


{'input': 'what is 2 + 2',
 'text': 'The answer to the question "what is 2 + 2" is 4. This is a basic addition problem in mathematics, where two numbers are added together to produce a sum. In this case, 2 + 2 equals 4.'}

In [21]:
chain.invoke ("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


{'input': 'Why does every cell in our body contain DNA?',
 'text': "DNA, or deoxyribonucleic acid, is the genetic material of all living organisms, including humans. It is responsible for storing genetic information and plays a crucial role in the growth, development, reproduction, and functioning of cells. Therefore, every cell in our body contains DNA.\n\nDNA molecules are composed of two chains of nucleotides twisted around each other to form a double helix structure. Each nucleotide contains a nitrogenous base, a deoxyribose sugar, and a phosphate group. The nitrogenous bases include adenine (A), guanine (G), cytosine (C), and thymine (T), which pair up in a specific way to form the genetic code.\n\nThe DNA in each cell contains all the genetic information needed for that cell to function properly. However, not all genes are expressed in every cell. Cells differentiate into different types by expressing different sets of genes, which allows them to perform specialized functions. Fo

## LangChain: Q&A over Documents


In [ ]:
#pip install docarray

In [36]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.indexes import VectorstoreIndexCreator

from langchain.embeddings import QianfanEmbeddingsEndpoint
from langchain.chat_models import QianfanChatEndpoint

from IPython.display import display, Markdown
# from langchain.llms import OpenAI


In [43]:
file = '..\_data\OutdoorClothingCatalog_999.csv'
loader = CSVLoader(file_path=file, encoding='utf-8')
# data = loader.load()
# print(data)

In [44]:
embeddings = QianfanEmbeddingsEndpoint()
index = VectorstoreIndexCreator(
    embedding=embeddings,
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [45]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."
llm_replacement_model = QianfanChatEndpoint(model="ERNIE-Bot-4", temperature = 0.01) 
response = index.query(query, 
                       llm = llm_replacement_model)

In [47]:
display(Markdown(response))

| Shirt Name | Fabric | SPF Rating | UV Protection | Summary |
| --- | --- | --- | --- | --- |
| Sun Guard Shirt | High-performance fabric | SPF 50+ | Blocks 98% of harmful UV rays | This shirt offers excellent sun protection with its high-performance fabric that blocks 98% of the sun's harmful UV rays. It provides an SPF 50+ rating, making it an ideal choice for outdoor activities. |

Note: Currently, we only have one shirt with sun protection available, but we are constantly updating our inventory. Please check back for more options soon.

The Sun Guard Shirt is our top pick for sun protection. Made from high-performance fabric, it effectively blocks 98% of the sun's harmful UV rays, providing an SPF 50+ rating. This shirt is perfect for those who spend a lot of time outdoors and want reliable protection from the sun's damaging rays. Whether you're at the beach, hiking, or just enjoying a sunny day, the Sun Guard Shirt will keep your skin safe and protected.

### Step By Step 

In [48]:
loader = CSVLoader(file_path=file, encoding='utf-8')
docs = loader.load()
print(len(docs)) 
docs[0]

999


Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.", metadata={'source': '..\\_data\\OutdoorClothingCatalog_999.csv', 'row': 0})

In [49]:
embeddings = QianfanEmbeddingsEndpoint()
embed = embeddings.embed_query("Hi my name is Harrison")
print(len(embed))
print(embed[:5])

384
[0.1582121104001999, 0.02535087615251541, 0.026235725730657578, 0.04640427231788635, -0.02119000069797039]


In [50]:
# 不知道为什么，这里会报错。
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)
docs = db.similarity_search(query)
query = "Please suggest a shirt with sunblocking"
docs = db.similarity_search(query)
print(len(docs), docs[0])


[ERROR] [05-04 18:03:47] openapi_requestor.py:246 [t:44652]: api request req_id: as-j0n2xr77cf failed with error code: 336003, err msg: embeddings max length per batch size is 1000, please check https://cloud.baidu.com/doc/WENXINWORKSHOP/s/tlmyncueh


APIError: api return error, req_id: as-j0n2xr77cf code: 336003, msg: embeddings max length per batch size is 1000

In [ ]:
retriever = db.as_retriever()

llm = QianfanChatEndpoint(model="ERNIE-Bot-4", temperature = 0.01) 
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 

display(Markdown(response))

In [ ]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

response = qa_stuff.run(query)

display(Markdown(response))

In [ ]:
response = index.query(query, llm=llm)

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

#### Combine examples

In [ ]:
examples += new_examples

In [ ]:
qa.run(examples[0]["query"])

### Manual Evaluation

In [ ]:
import langchain
langchain.debug = True

In [ ]:
qa.run(examples[0]["query"])

In [ ]:
# Turn off the debug mode
langchain.debug = False

## 基于文档QA问答系统

包含更多说明的，参见 [question_answering_for_qianfan_langchain.ipynb](../qianfan-langchain-QA-demo/question_answering_for_qianfan_langchain.ipynb)

In [9]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import QianfanEmbeddingsEndpoint
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import QianfanChatEndpoint
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain


### Prepare

In [2]:
WEB_URL = "https://zhuanlan.zhihu.com/p/85289282"
WEB_URL =  "https://xuxiangwen.github.io/others/2022/04/15/sh_coronavirus_log/"

CUSTOM_PROMPT_TEMPLATE = """
使用下面的语料来回答本模板最末尾的问题。如果你不知道问题的答案，直接回答 "我不知道"，禁止随意编造答案。
为了保证答案尽可能简洁，你的回答必须不超过三句话。
请注意！在每次回答结束之后，你都必须接上 "感谢你的提问" 作为结束语
以下是一对问题和答案的样例：
    请问：秦始皇的原名是什么
    秦始皇原名嬴政。感谢你的提问。

以下是语料：

{context}

请问：{question}
"""
QUESTION1 = "明朝的开国皇帝是谁"
QUESTION2 = "朱元璋是什么时候建立的明朝"
QUESTION3 = "朱元璋有几个儿子"
QUESTION4 = "朱元璋的孙子是谁"

QUESTION1 = "我是什么时候打的第二针疫苗的"
QUESTION2 = "我是什么时候抗原检测阳性"
QUESTION3 = "球队的大腿在方舱的什么地方"
QUESTION4 = "方舱的伙食有哪些"


### Load

In [3]:

loader = WebBaseLoader(WEB_URL)
data = loader.load()
print(type(data))
print(type(data[0]))

<class 'list'>
<class 'langchain_core.documents.base.Document'>


In [4]:
print(len(data[0].page_content)) 
print(data[0].page_content)

4638






新冠病毒找上了我 | Weclome to eipi10



























Weclome to eipi10




View On GitHub






新冠病毒找上了我

  15 Apr 2022

2022年4月上海，我感染了新冠病毒，本文记录了从在家发病，转运方舱，以及方舱回家的一段生活。非常感谢家人，朋友和同学这段时间对我的关心和支持，这让我感到有了无穷的力量。
疫苗

本人：两针
    
2022-04-02 ：北京生物疫苗第一针，
2022-04-04：北京生物疫苗第二针


孩子：两针，2022年3月份打的第二针
母亲和妻子：三针

2022-03-26 周六 家

自测抗原，全家阴性
和本楼志愿者有接触，他们家四人当天抗原阳性，当晚核酸检测都是阳性。估计被这个传染的可能性很大
楼栋单元被封。

2022-03-28 周一 家

全家核酸检测。这个核酸检测结果一直未出。

2022-03-30 周三 家

喉咙有点痒, 有一点头晕和乏力。
全家核酸检测，第二天结果出来，都是阴性

2022-03-31 周四 家

喉咙发干；有些头晕，全身乏力
大部分时间卧床休息。
吃阿咖酚散。
晚上工作了两个小时。

2022-04-01 周五 家


喉咙非常痛，像针扎；有一些发烧（38度左右）；头晕，全身无力；嗓子有些哑。


卧床休息。


吃阿咖酚散，下午开始吃连花清瘟胶囊。小区的球队兄弟还给我带了两盒，谢谢啊。


自测抗原阳性。家人都是阴性，且没有任何异样。



开始被隔离在家里单独房间。


祸不单行，电脑不知道怎么被水泼了，无法启动。


2022-04-02 周六 家


喉咙非常痛，像针扎；有一些发烧（38度左右）；头晕，全身无力；嗓子非常哑。


卧床休息。


吃连花清瘟胶囊。


个人核酸检测。这个核酸检测结果一直未出。


飘窗晒电脑。试了很多次，无法开机，晚上睡觉前，又试了一下，启动了。


女儿送我千纸鹤。



2022-04-03 周日 家

喉咙不太痛了；体温37度左右；有时会咳黄痰；有轻微鼻塞。
一半时间卧床，可以玩游戏了。
吃连花清瘟胶囊。

2022-04-04 周一 家


喉咙完全不痛了，只是有些发干；咳嗽变得很厉害，痰多，黄白色；鼻塞，鼻涕多。



### Split

In [5]:


# 建议chunk_size限制在400以内
# separator里面的顺序是有优先级别的，最后一个必须是""
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 384, chunk_overlap = 0, separators=["。", "，", "\n\n", "\n", " ", ""])
all_splits = text_splitter.split_documents(data)
print(type(all_splits))
print(type(all_splits[0]))

<class 'list'>
<class 'langchain_core.documents.base.Document'>


In [6]:
print(all_splits[0].page_content)
print(len(all_splits[0].page_content))

新冠病毒找上了我 | Weclome to eipi10



























Weclome to eipi10




View On GitHub






新冠病毒找上了我

  15 Apr 2022

2022年4月上海，我感染了新冠病毒，本文记录了从在家发病，转运方舱，以及方舱回家的一段生活。非常感谢家人，朋友和同学这段时间对我的关心和支持，这让我感到有了无穷的力量。
疫苗

本人：两针
    
2022-04-02 ：北京生物疫苗第一针，
2022-04-04：北京生物疫苗第二针


孩子：两针，2022年3月份打的第二针
母亲和妻子：三针

2022-03-26 周六 家

自测抗原，全家阴性
和本楼志愿者有接触，他们家四人当天抗原阳性，当晚核酸检测都是阳性
360


In [7]:
print(all_splits[1].page_content)
print(len(all_splits[1].page_content))

。估计被这个传染的可能性很大
楼栋单元被封。

2022-03-28 周一 家

全家核酸检测。这个核酸检测结果一直未出。

2022-03-30 周三 家

喉咙有点痒, 有一点头晕和乏力。
全家核酸检测，第二天结果出来，都是阴性

2022-03-31 周四 家

喉咙发干；有些头晕，全身乏力
大部分时间卧床休息。
吃阿咖酚散。
晚上工作了两个小时。

2022-04-01 周五 家


喉咙非常痛，像针扎；有一些发烧（38度左右）；头晕，全身无力；嗓子有些哑。


卧床休息。


吃阿咖酚散，下午开始吃连花清瘟胶囊。小区的球队兄弟还给我带了两盒，谢谢啊。


自测抗原阳性。家人都是阴性，且没有任何异样。



开始被隔离在家里单独房间。


祸不单行，电脑不知道怎么被水泼了，无法启动
349


In [8]:
print(all_splits[2].page_content)
print(len(all_splits[2].page_content))

。


2022-04-02 周六 家


喉咙非常痛，像针扎；有一些发烧（38度左右）；头晕，全身无力；嗓子非常哑。


卧床休息。


吃连花清瘟胶囊。


个人核酸检测。这个核酸检测结果一直未出。


飘窗晒电脑。试了很多次，无法开机，晚上睡觉前，又试了一下，启动了。


女儿送我千纸鹤。



2022-04-03 周日 家

喉咙不太痛了；体温37度左右；有时会咳黄痰；有轻微鼻塞。
一半时间卧床，可以玩游戏了。
吃连花清瘟胶囊。

2022-04-04 周一 家


喉咙完全不痛了，只是有些发干；咳嗽变得很厉害，痰多，黄白色；鼻塞，鼻涕多。

下午开始连续喷嚏，痰基本是白色。体温36.4度



精力感觉基本恢复了。


吃连花清瘟胶囊。


自测抗原阳性。



女儿送我福袋
348


### Store

In [9]:

embeddings = QianfanEmbeddingsEndpoint()

In [10]:
embed = embeddings.embed_query(QUESTION1)

[INFO] [04-30 19:24:34] oauth.py:222 [t:40792]: trying to refresh access_token for ak `HCCPsQ***`
[INFO] [04-30 19:24:35] oauth.py:237 [t:40792]: sucessfully refresh access_token


In [11]:
print(len(embed))

384


In [12]:
print(embed[:5])

[0.05926443636417389, 0.048030029982328415, 0.035678666085004807, -0.020849037915468216, 0.004837945103645325]


In [13]:
#!pip install chromadb

In [14]:
vectorstore = Chroma.from_documents(documents=all_splits, embedding=embeddings)

### Retrieve

In [15]:
print("prompt问题："+QUESTION1)
docs = vectorstore.similarity_search_with_relevance_scores(QUESTION1)
[(document.page_content, score) for document, score in docs]

prompt问题：我是什么时候打的第二针疫苗的


[('新冠病毒找上了我 | Weclome to eipi10\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nWeclome to eipi10\n\n\n\n\nView On GitHub\n\n\n\n\n\n\n新冠病毒找上了我\n\n  15 Apr 2022\n\n2022年4月上海，我感染了新冠病毒，本文记录了从在家发病，转运方舱，以及方舱回家的一段生活。非常感谢家人，朋友和同学这段时间对我的关心和支持，这让我感到有了无穷的力量。\n疫苗\n\n本人：两针\n    \n2022-04-02 ：北京生物疫苗第一针，\n2022-04-04：北京生物疫苗第二针\n\n\n孩子：两针，2022年3月份打的第二针\n母亲和妻子：三针\n\n2022-03-26 周六 家\n\n自测抗原，全家阴性\n和本楼志愿者有接触，他们家四人当天抗原阳性，当晚核酸检测都是阳性',
  0.3107365085990246),
 ('。真的搞不清了，我究竟啥时候阳的，为啥又可以出了。我哥说他能查到核酸记录，我却不能，目前好像就是方舱的人查不到核酸检查记录。今天早上又进行了核酸，真的有些担心，不能高兴太早了啊。希望方舱早日清空，上海早日清零，大家早日回归正常的生活。\n\n\n\n家里的小猫怎么这么可爱。\n\n\n\n2022-04-15 周五 方舱\n\n\n早上6:00护士给我又发了出舱的贴牌。到周围转了一圈，发现也有人被通知不能出舱，看来阴阳互换，还是有不少的。\n\n\n\n\n今天要出舱了，排队中。这次应该没跑了，发了通知给家人，同学和朋友，让他们安心。这次方舱经历，收获了太多的祝福和支持，诚挚的感谢大家。\n\n\n\n坐上了公交车，就要离开住了10晚的方舱了，心里忽然有一丝的不舍，这真的是一段很特别的经历啊。不到20分钟，车开到了中转站——源深体育馆',
  0.07587538690819018),
 ('。\n\n\n今天突破三公里了。以后每天保持这个量啦。估计这样下去，体重可以再降一降。\n\n\n\n今天配了酸奶，加上早上剩下的牛奶，还不错。\n\n\n\n今天老婆说，她去我单独居家隔离的房间拿东西，不想“西西”也偷偷跟进去了，不得已只好给小猫喷点酒精消毒，然后洗澡了。“西西”是不是也想我

In [17]:
print("prompt问题："+QUESTION2)
docs = vectorstore.similarity_search_with_relevance_scores(QUESTION2)
[(document.page_content, score) for document, score in docs]

prompt问题：我是什么时候抗原检测阳性


[('新冠病毒找上了我 | Weclome to eipi10\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nWeclome to eipi10\n\n\n\n\nView On GitHub\n\n\n\n\n\n\n新冠病毒找上了我\n\n  15 Apr 2022\n\n2022年4月上海，我感染了新冠病毒，本文记录了从在家发病，转运方舱，以及方舱回家的一段生活。非常感谢家人，朋友和同学这段时间对我的关心和支持，这让我感到有了无穷的力量。\n疫苗\n\n本人：两针\n    \n2022-04-02 ：北京生物疫苗第一针，\n2022-04-04：北京生物疫苗第二针\n\n\n孩子：两针，2022年3月份打的第二针\n母亲和妻子：三针\n\n2022-03-26 周六 家\n\n自测抗原，全家阴性\n和本楼志愿者有接触，他们家四人当天抗原阳性，当晚核酸检测都是阳性',
  0.2030041904770552),
 ('。估计被这个传染的可能性很大\n楼栋单元被封。\n\n2022-03-28 周一 家\n\n全家核酸检测。这个核酸检测结果一直未出。\n\n2022-03-30 周三 家\n\n喉咙有点痒, 有一点头晕和乏力。\n全家核酸检测，第二天结果出来，都是阴性\n\n2022-03-31 周四 家\n\n喉咙发干；有些头晕，全身乏力\n大部分时间卧床休息。\n吃阿咖酚散。\n晚上工作了两个小时。\n\n2022-04-01 周五 家\n\n\n喉咙非常痛，像针扎；有一些发烧（38度左右）；头晕，全身无力；嗓子有些哑。\n\n\n卧床休息。\n\n\n吃阿咖酚散，下午开始吃连花清瘟胶囊。小区的球队兄弟还给我带了两盒，谢谢啊。\n\n\n自测抗原阳性。家人都是阴性，且没有任何异样。\n\n\n\n开始被隔离在家里单独房间。\n\n\n祸不单行，电脑不知道怎么被水泼了，无法启动',
  0.1644475791097132),
 ('。\n\n\n2022-04-02 周六 家\n\n\n喉咙非常痛，像针扎；有一些发烧（38度左右）；头晕，全身无力；嗓子非常哑。\n\n\n卧床休息。\n\n\n吃连花清瘟胶囊。\n\n\n个人核酸检测。这个核酸检测结果一直未出。\n\n\n飘窗晒电脑。试了很多次

In [18]:
print("prompt问题："+QUESTION3)
docs = vectorstore.similarity_search_with_relevance_scores(QUESTION3) 
[(document.page_content, score) for document, score in docs]

prompt问题：球队的大腿在方舱的什么地方


[('。当了一下志愿者，帮忙大白来维护核酸的队伍。哪一个是我？\n\n\n\n闲逛\n\n\n\n下午溜到W4和球队的大腿相见。\n\n\n\n经过了几天的磨合，方舱的环境也越来越好。开始的时候，大家把东西乱扔，水槽垃圾尤其多，厕所很脏，现在，大家越来越爱护自己生活的环境了，垃圾少了，上厕所的时候，很多人都会准备一盆水。\n\n\n2022-04-10 周日 方舱\n\n\n今天早上总算能跑起来了。\n\n\n\n闲逛\n\n\n\n又有百叶卷？好像是上海清明特产。\n\n\n\n昨天下午，护士过来通知有一部分人，明天可以回家了。瞄了一眼名单，大概有10%左右。我的对床也被通知了，什么时候才轮到我。\n\n\n\n这几天陆陆续续看到很多负面的消息。上海作为超大城市，物资供应和医疗服务都是极其严峻的考验，现在正是困难时刻，上海坚持住。\n\n\n2022-04-11 周一 方舱\n\n\n或许是对床今天就要回家了，今天忽然很想回家',
  0.16612839542832758),
 ('。\n\n\n早上6点多，护士来通知我，说不能出舱了，核酸复阳，收回了我的贴牌，心里真的好失落。外面也一直下着雨，就像我的心情。之前球队大腿说，如果你是最后10%出舱的，你就牛了，看来他的乌鸦嘴要灵验了。\n\n\n今天的雨很大，屋顶轰隆隆超级响。网上看到其它地方的方舱漏雨，我们这算很好了。\n\n\n\n下午在整理方舱的涂鸦集，把图片变小，整理好后，文件从300MB变成了不到100MB。本来想今天能出去的，想把这个涂鸦集作为方舱的留念，毕竟这个经历也挺难得的。看来老天是想让我多收集一些啊，明天再去转转，看看还有什么新的涂鸦。\n\n\n2022-04-14 周四 方舱\n\n\n早上起来，还有小雨，没有跑步，也没有去做志愿者，想想还是好好休息一下吧。\n\n\n上午基本不下雨了，天气也凉爽起来，感觉心情突然好起来了。\n\n\n中午，护士突然过来通知，说明天可以出舱',
  0.1182478053523145),
 ('。\n\n\n今天突破三公里了。以后每天保持这个量啦。估计这样下去，体重可以再降一降。\n\n\n\n今天配了酸奶，加上早上剩下的牛奶，还不错。\n\n\n\n今天老婆说，她去我单独居家隔离的房间拿东西，不想“西西”也偷偷跟进去了，不得已只好给小猫喷点酒精消毒，然

In [19]:
print("prompt问题："+QUESTION4)
docs = vectorstore.similarity_search_with_relevance_scores(QUESTION4) 
[(document.page_content, score) for document, score in docs]

prompt问题：方舱的伙食有哪些


[('。\n\n\n早上散步\n\n\n\n连了wifi，网速也快起来了，嗖嗖的。要来串羊肉串吗？\n\n\n\n\u200b\t\t不过上午9点后，就明显慢下来了，看来还是人多。\n\n\n晚上11:00多，开始测核酸了。上海能使用的方舱已经8w+，在建的还有几w，但仍然不够用啊。看到下面这个通知，看来尽快出舱有希望了。\n\n\n\n2022-04-08 周五 方舱\n\n\n还有偶尔有一点咳嗽和痰。看来这个状态将会持续一段时间啦\n\n\n早上5点多，听到孩子连续撕裂的哭声，估计是不舒服喉咙痛。这哭声提醒我，虽然大多数人是轻症甚至无症状，但由于感染人数巨大，少数的重症其实绝对数量也不小，绝不可掉以轻心。\n\n\n哥哥发了他们附近小区的一些”唱歌”的画面。\n的确有不少人家里已经没粮了。或许能够在方舱也是一种幸运。\n\n\n2022-04-09 周六 方舱\n\n\n今天有大鸡腿。\n\n\n\n从7号开始一天一次核酸',
  0.20967308631693393),
 ('。\n\n\n今天突破三公里了。以后每天保持这个量啦。估计这样下去，体重可以再降一降。\n\n\n\n今天配了酸奶，加上早上剩下的牛奶，还不错。\n\n\n\n今天老婆说，她去我单独居家隔离的房间拿东西，不想“西西”也偷偷跟进去了，不得已只好给小猫喷点酒精消毒，然后洗澡了。“西西”是不是也想我了啊？\n\n\n\n2022-04-12 周二 方舱\n\n\n早上帮着分发早餐，分了40分钟，还是有点儿累啊。由此，今天早上只跑了1.5公里。\n\n\n上午给武汉第一医院的医护拍了些照片和视频。拍完后，他们自己剪了一个视频，效果还不错啊。\n\n\n\n中午护士通知我明天可以出舱了，非常高兴啊。对床的昨天没有出去，也是明天，明天好多人都可以出舱了。发的贴牌，明天贴衣服上。\n\n\n\n2022-04-13 周三 方舱\n\n\n早上五点就起来了，早餐分发了一个小时。下雨，没有跑步',
  0.20451844245035455),
 ('。\n\n\n分发的东西很齐备。\n\n\n\n\n\n\n2022-04-06 周三 方舱\n\n\n早上补了一觉，咳嗽痰都少了好多，鼻涕基本没了，越来越好了。\n\n\n白天是10院的医护人员，晚上换岗了，是武汉来支援的医护人员。\n\n\n\n吃清肺

### Generate

In [20]:

QA_CHAIN_PROMPT = PromptTemplate.from_template(CUSTOM_PROMPT_TEMPLATE)

llm = QianfanChatEndpoint(model="ERNIE-Bot-4", streaming=True)   # 
retriever = vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={'score_threshold': 0.0})
                                   
qa_chain = RetrievalQA.from_chain_type(llm, retriever = retriever, 
                                       chain_type = "stuff",
                                       chain_type_kwargs = {"prompt": QA_CHAIN_PROMPT}, 
                                       verbose = True,
                                       return_source_documents = True)
result = qa_chain({"query": QUESTION1})

C:\Users\xu6\AppData\Local\miniconda3\envs\dev\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.


In [21]:
QUESTION = QUESTION1
result = qa_chain({"query": QUESTION})
print(f'QUESTION1 = {QUESTION}')
print(len(result['source_documents']))
print(result['source_documents'])
result["result"]



> Entering new RetrievalQA chain...

> Finished chain.
QUESTION1 = 我是什么时候打的第二针疫苗的
4
[Document(page_content='新冠病毒找上了我 | Weclome to eipi10\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nWeclome to eipi10\n\n\n\n\nView On GitHub\n\n\n\n\n\n\n新冠病毒找上了我\n\n  15 Apr 2022\n\n2022年4月上海，我感染了新冠病毒，本文记录了从在家发病，转运方舱，以及方舱回家的一段生活。非常感谢家人，朋友和同学这段时间对我的关心和支持，这让我感到有了无穷的力量。\n疫苗\n\n本人：两针\n    \n2022-04-02 ：北京生物疫苗第一针，\n2022-04-04：北京生物疫苗第二针\n\n\n孩子：两针，2022年3月份打的第二针\n母亲和妻子：三针\n\n2022-03-26 周六 家\n\n自测抗原，全家阴性\n和本楼志愿者有接触，他们家四人当天抗原阳性，当晚核酸检测都是阳性', metadata={'description': '2022年4月上海，我感染了新冠病毒，本文记录了从在家发病，转运方舱，以及方舱回家的一段生活。非常感谢家人，朋友和同学这段时间对我的关心和支持，这让我感到有了无穷的力量。', 'language': 'en-US', 'source': 'https://xuxiangwen.github.io/others/2022/04/15/sh_coronavirus_log/', 'title': '新冠病毒找上了我 | Weclome to eipi10'}), Document(page_content='。真的搞不清了，我究竟啥时候阳的，为啥又可以出了。我哥说他能查到核酸记录，我却不能，目前好像就是方舱的人查不到核酸检查记录。今天早上又进行了核酸，真的有些担心，不能高兴太早了啊。希望方舱早日清空，上海早日清零，大家早日回归正常的生活。\n\n\n\n家里的小猫怎么这么可爱。\n\n\n\n2022-04-15 周五 方舱\n\n\n早上6:00护士给我又发了出舱的贴牌。到

'你是在2022年4月4日打的第二针疫苗。感谢你的提问。'

In [22]:
QUESTION = QUESTION2
result = qa_chain({"query": QUESTION})
print(f'QUESTION1 = {QUESTION}')
print(len(result['source_documents']))
print(result['source_documents'])
result["result"]



> Entering new RetrievalQA chain...

> Finished chain.
QUESTION1 = 我是什么时候抗原检测阳性
4
[Document(page_content='新冠病毒找上了我 | Weclome to eipi10\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nWeclome to eipi10\n\n\n\n\nView On GitHub\n\n\n\n\n\n\n新冠病毒找上了我\n\n  15 Apr 2022\n\n2022年4月上海，我感染了新冠病毒，本文记录了从在家发病，转运方舱，以及方舱回家的一段生活。非常感谢家人，朋友和同学这段时间对我的关心和支持，这让我感到有了无穷的力量。\n疫苗\n\n本人：两针\n    \n2022-04-02 ：北京生物疫苗第一针，\n2022-04-04：北京生物疫苗第二针\n\n\n孩子：两针，2022年3月份打的第二针\n母亲和妻子：三针\n\n2022-03-26 周六 家\n\n自测抗原，全家阴性\n和本楼志愿者有接触，他们家四人当天抗原阳性，当晚核酸检测都是阳性', metadata={'description': '2022年4月上海，我感染了新冠病毒，本文记录了从在家发病，转运方舱，以及方舱回家的一段生活。非常感谢家人，朋友和同学这段时间对我的关心和支持，这让我感到有了无穷的力量。', 'language': 'en-US', 'source': 'https://xuxiangwen.github.io/others/2022/04/15/sh_coronavirus_log/', 'title': '新冠病毒找上了我 | Weclome to eipi10'}), Document(page_content='。估计被这个传染的可能性很大\n楼栋单元被封。\n\n2022-03-28 周一 家\n\n全家核酸检测。这个核酸检测结果一直未出。\n\n2022-03-30 周三 家\n\n喉咙有点痒, 有一点头晕和乏力。\n全家核酸检测，第二天结果出来，都是阴性\n\n2022-03-31 周四 家\n\n喉咙发干；有些头晕，全身乏力\n大部分时间卧床休息。\n吃阿咖酚散。\n晚上工作

'你在2022年4月1日自测抗原阳性。感谢你的提问。'

In [23]:
QUESTION = QUESTION3
result = qa_chain({"query": QUESTION})
print(f'QUESTION1 = {QUESTION}')
print(len(result['source_documents']))
print(result['source_documents'])
result["result"]



> Entering new RetrievalQA chain...

> Finished chain.
QUESTION1 = 球队的大腿在方舱的什么地方
4
[Document(page_content='。当了一下志愿者，帮忙大白来维护核酸的队伍。哪一个是我？\n\n\n\n闲逛\n\n\n\n下午溜到W4和球队的大腿相见。\n\n\n\n经过了几天的磨合，方舱的环境也越来越好。开始的时候，大家把东西乱扔，水槽垃圾尤其多，厕所很脏，现在，大家越来越爱护自己生活的环境了，垃圾少了，上厕所的时候，很多人都会准备一盆水。\n\n\n2022-04-10 周日 方舱\n\n\n今天早上总算能跑起来了。\n\n\n\n闲逛\n\n\n\n又有百叶卷？好像是上海清明特产。\n\n\n\n昨天下午，护士过来通知有一部分人，明天可以回家了。瞄了一眼名单，大概有10%左右。我的对床也被通知了，什么时候才轮到我。\n\n\n\n这几天陆陆续续看到很多负面的消息。上海作为超大城市，物资供应和医疗服务都是极其严峻的考验，现在正是困难时刻，上海坚持住。\n\n\n2022-04-11 周一 方舱\n\n\n或许是对床今天就要回家了，今天忽然很想回家', metadata={'description': '2022年4月上海，我感染了新冠病毒，本文记录了从在家发病，转运方舱，以及方舱回家的一段生活。非常感谢家人，朋友和同学这段时间对我的关心和支持，这让我感到有了无穷的力量。', 'language': 'en-US', 'source': 'https://xuxiangwen.github.io/others/2022/04/15/sh_coronavirus_log/', 'title': '新冠病毒找上了我 | Weclome to eipi10'}), Document(page_content='。\n\n\n早上6点多，护士来通知我，说不能出舱了，核酸复阳，收回了我的贴牌，心里真的好失落。外面也一直下着雨，就像我的心情。之前球队大腿说，如果你是最后10%出舱的，你就牛了，看来他的乌鸦嘴要灵验了。\n\n\n今天的雨很大，屋顶轰隆隆超级响。网上看到其它地方的方舱漏雨，我们这算很好了。\n\n\n\n下午在整理方舱的涂鸦集，把图片变小，整理好后，文件从300MB变成了不到

'很抱歉，文中没有提及球队的大腿在方舱的具体位置。感谢你的提问。'

In [24]:
QUESTION = QUESTION4
result = qa_chain({"query": QUESTION})
print(f'QUESTION1 = {QUESTION}')
print(len(result['source_documents']))
print(result['source_documents'])
result["result"]



> Entering new RetrievalQA chain...

> Finished chain.
QUESTION1 = 方舱的伙食有哪些
4
[Document(page_content='。\n\n\n早上散步\n\n\n\n连了wifi，网速也快起来了，嗖嗖的。要来串羊肉串吗？\n\n\n\n\u200b\t\t不过上午9点后，就明显慢下来了，看来还是人多。\n\n\n晚上11:00多，开始测核酸了。上海能使用的方舱已经8w+，在建的还有几w，但仍然不够用啊。看到下面这个通知，看来尽快出舱有希望了。\n\n\n\n2022-04-08 周五 方舱\n\n\n还有偶尔有一点咳嗽和痰。看来这个状态将会持续一段时间啦\n\n\n早上5点多，听到孩子连续撕裂的哭声，估计是不舒服喉咙痛。这哭声提醒我，虽然大多数人是轻症甚至无症状，但由于感染人数巨大，少数的重症其实绝对数量也不小，绝不可掉以轻心。\n\n\n哥哥发了他们附近小区的一些”唱歌”的画面。\n的确有不少人家里已经没粮了。或许能够在方舱也是一种幸运。\n\n\n2022-04-09 周六 方舱\n\n\n今天有大鸡腿。\n\n\n\n从7号开始一天一次核酸', metadata={'description': '2022年4月上海，我感染了新冠病毒，本文记录了从在家发病，转运方舱，以及方舱回家的一段生活。非常感谢家人，朋友和同学这段时间对我的关心和支持，这让我感到有了无穷的力量。', 'language': 'en-US', 'source': 'https://xuxiangwen.github.io/others/2022/04/15/sh_coronavirus_log/', 'title': '新冠病毒找上了我 | Weclome to eipi10'}), Document(page_content='。\n\n\n今天突破三公里了。以后每天保持这个量啦。估计这样下去，体重可以再降一降。\n\n\n\n今天配了酸奶，加上早上剩下的牛奶，还不错。\n\n\n\n今天老婆说，她去我单独居家隔离的房间拿东西，不想“西西”也偷偷跟进去了，不得已只好给小猫喷点酒精消毒，然后洗澡了。“西西”是不是也想我了啊？\n\n\n\n2022-04-12 周二 方舱\n\n\n早上帮着分发早餐，分了40分钟，

'方舱的伙食包括肉类、蔬菜和酸奶等，具体可能会有大鸡腿、百叶卷等食物。感谢你的提问。'

### Chat

In [25]:


memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory, combine_docs_chain_kwargs={"prompt": QA_CHAIN_PROMPT})
qa(QUESTION1)

{'question': '我是什么时候打的第二针疫苗的',
 'chat_history': [SystemMessage(content='')],
 'answer': '你第二针疫苗是在2022年4月4日接种的。感谢你的提问。'}

In [26]:
qa(QUESTION2)

{'question': '我是什么时候抗原检测阳性',
 'chat_history': [SystemMessage(content='人类询问自己第二针疫苗的接种时间，AI回答称是在2022年4月4日接种的。')],
 'answer': '你是在2022年4月1日自测抗原阳性的。感谢你的提问。'}

In [27]:
qa(QUESTION3)

{'question': '球队的大腿在方舱的什么地方',
 'chat_history': [SystemMessage(content='人类询问自己第二针疫苗的接种时间，AI回答称是在2022年4月4日接种的。接着，人类又问自己是什么时候抗原检测阳性，AI回答是在2022年4月1日自测抗原阳性的。')],
 'answer': '我不知道球队的大腿在方舱的什么地方，因为原文中并没有提到相关信息。感谢你的提问。'}

In [28]:
qa(QUESTION4)

{'question': '方舱的伙食有哪些',
 'chat_history': [SystemMessage(content='人类询问自己第二针疫苗的接种时间以及自测抗原阳性的时间，AI分别回答是在2022年4月4日和2022年4月1日。接着，人类问了一个与方舱和球队大腿相关的问题，但AI表示原文中没有提到相关信息，无法回答。')],
 'answer': '方舱的伙食通常包括营养丰富的午餐和晚餐，有大鸡腿、百叶卷等食物。感谢你的提问。'}